# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!rm -rf ../_publications/*md
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url
2024-03-24	The Origin and Evolution of Sex Peptide and Sex Peptide Receptor Interactions	Molecular Biology and Evolution	Description	<b>Junhui Peng</b>, Nicolas Svetec, Henrik Molina, Li Zhao: The Origin and Evolution of Sex Peptide and Sex Peptide Receptor Interactions. <b>Mol Biol Evol</b> 2024; 41(4).	paper-title-number-1	<a href="https://academic.oup.com/mbe/article/41/4/msae065/7633838">Link</a> <a href="https://jhpanda.github.io/files/sp.pdf">PDF</a>
2024-01-27	The origin and structural evolution of de novo genes in Drosophila	Nature Communications	Description	<b>Junhui Peng</b>, Li Zhao: The origin and structural evolution of de novo genes in Drosophila. <b>Nat Commun</b> 2024; 15, 810.	paper-title-number-1	<a href="https://www.nature.com/articles/s41467-024-45028-1">Link</a> <a href="https://jhpanda.github.io/files/denovo_genes.pdf">PDF</a>
2021-12-08	Intermolecular interactions drive protein adaptive and co-adaptive evol

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2024-03-24,The Origin and Evolution of Sex Peptide and Se...,Molecular Biology and Evolution,Description,"<b>Junhui Peng</b>, Nicolas Svetec, Henrik Mol...",paper-title-number-1,"<a href=""https://academic.oup.com/mbe/article/..."
1,2024-01-27,The origin and structural evolution of de novo...,Nature Communications,Description,"<b>Junhui Peng</b>, Li Zhao: The origin and st...",paper-title-number-1,"<a href=""https://www.nature.com/articles/s4146..."
2,2021-12-08,Intermolecular interactions drive protein adap...,Molecular Biology and Evolution,Intermolecular interactions drive protein adap...,"<b>Junhui Peng</b>, Nicolas Svetec, Li Zhao, I...",paper-title-number-2,"<a href=""https://academic.oup.com/mbe/article/..."
3,2021-02-15,Behavioral and Genomic Sensory Adaptations Und...,Molecular Biology and Evolution,Description,"Sylvia M Durkin, Mahul Chakraborty, Antoine A...",paper-title-number-3,"<a href=""https://academic.oup.com/mbe/article/..."
4,2020-07-14,Molecular mechanism of histone variant H2A. B ...,Epigenetics & chromatin,Description,"<b>Junhui Peng</b>, Chuang Yuan, Xinfan Hua an...",paper-title-number-4,"<a href=""https://epigeneticsandchromatin.biome..."
5,2019-03-25,Backmapping from Multiresolution Coarse-Graine...,Journal of Chemical Theory and Computation,Description,"<b>Junhui Peng</b>, Chuang Yuan, Rongsheng Ma,...",paper-title-number-5,"<a href=""https://pubs.acs.org/doi/abs/10.1021/..."
6,2018-09-25,Clustering algorithms to analyze molecular dyn...,Chinese Journal of Chemical Physics,Description,"<b>Junhui Peng</b>, Wei Wang, Yeqing Yu, Hanli...",paper-title-number-6,"<a href=""https://cps.scitation.org/doi/10.1063..."
7,2017-12-05,Insights into the Structure of Dimeric RNA Hel...,Structure,Description,"Ling Xu, Lijun Wang, <b>Junhui Peng</b>, Fudon...",paper-title-number-7,"<a href=""https://www.sciencedirect.com/science..."


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
import os
idx = 1
for row, item in publications.iterrows():
    url_slug = "paper-title-number-%d"%idx
    md_filename = str(item.pub_date) + "-" + url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + url_slug
    year = item.pub_date[:4]
    idx += 1
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [6]:
!ls ../_publications/

2017-12-05-paper-title-number-8.md 2021-02-15-paper-title-number-4.md
2018-09-25-paper-title-number-7.md 2021-12-08-paper-title-number-3.md
2019-03-25-paper-title-number-6.md 2024-01-27-paper-title-number-2.md
2020-07-14-paper-title-number-5.md 2024-03-24-paper-title-number-1.md


In [7]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

cat: ../_publications/2009-10-01-paper-title-number-1.md: No such file or directory
